# Prueba Celes


## SQL 

**Modelo de datos**
![](C:/Users/kaes1/Pictures/Screenshots/Diagrama.png)

1. Crear una consulta que liste todos los enemigos que aparecen en los episodios de Doctor Who donde aparece Rose Tyler como compañero, pero no aparece David Tennant como el doctor. 

La consulta propuesta para listar los enemigos es la siguiente: 

In [ ]:
SELECT EnemyName
FROM `prueba_celes.tblEpisode` AS tblEpisode
INNER JOIN `prueba_celes.tblEpisodeCompanion` as tblEpisodeCompanion 
ON tblEpisode.EpisodeId = tblEpisodeCompanion.EpisodeId
INNER JOIN `prueba_celes.tblCompanion` as tblCompanion 
ON tblEpisodeCompanion.CompanionId = tblCompanion.CompanionId
INNER JOIN `prueba_celes.tblDoctor` as tblDoctor 
ON tblEpisode.DoctorId = tblDoctor.DoctorId
INNER JOIN `prueba_celes.tblEpisodeEnemy` as tblEpisodeEnemy 
ON tblEpisode.EpisodeId = tblEpisodeEnemy.EpisodeId
INNER JOIN `prueba_celes.tblEnemy` as tblEnemy
ON tblEpisodeEnemy.EnemyId = tblEnemy.EnemyId
WHERE CompanionName = "Rose Tyler" AND DoctorName != "David Tennant"
GROUP BY EnemyName

Luego de ejecutar la consulta, la lista de enemigos obtenidas fue: 

![](C:/Users/kaes1/Pictures/Screenshots/enemy.png)

2. Para cada episodio de David Tennant (como el doctor), muestre para cada enemigo el numero de veces que aparecen en episodios que no son de David. (La interpretación que se hizo de este query )

In [ ]:
SELECT tblEnemy.EnemyName, COUNT(tblEpisode.EpisodeId) AS NumApariciones
FROM `prueba_celes.tblEpisode` AS tblEpisode
INNER JOIN `prueba_celes.tblEpisodeEnemy` AS tblEpisodeEnemy ON tblEpisode.EpisodeId = tblEpisodeEnemy.EpisodeId
INNER JOIN `prueba_celes.tblEnemy` as tblEnemy ON tblEpisodeEnemy.EnemyId = tblEnemy.EnemyId
INNER JOIN `prueba_celes.tblDoctor` as tblDoctor ON tblEpisode.DoctorId = tblDoctor.DoctorId
WHERE DoctorName != "David Tennant" AND tblEnemy.EnemyId IN (SELECT tblEnemy.EnemyId
FROM `prueba_celes.tblEpisode` AS tblEpisode
INNER JOIN `prueba_celes.tblEpisodeEnemy` AS tblEpisodeEnemy ON tblEpisode.EpisodeId = tblEpisodeEnemy.EpisodeId
INNER JOIN `prueba_celes.tblEnemy` as tblEnemy ON tblEpisodeEnemy.EnemyId = tblEnemy.EnemyId
INNER JOIN `prueba_celes.tblDoctor` as tblDoctor ON tblEpisode.DoctorId = tblDoctor.DoctorId
WHERE DoctorName = "David Tennant"
GROUP BY EnemyId)
GROUP BY EnemyName

A continuación la lista de enemigos que aparecieron en capitulos con David Tennat, y la cantidad de veces que aparecieron en otros capitulos donde David Tennat no era el Doctor: 

![](C:/Users/kaes1/Pictures/Screenshots/enemy2.png)

3. Crea una consulta que liste los capitulos de David Tennat que no comparten enemigos con ningun otro episodio que no sea de Tennant.


In [ ]:
SELECT tblEpisode.EpisodeId, tblEpisode.Title
FROM `prueba_celes.tblEpisode` AS tblEpisode
INNER JOIN `prueba_celes.tblEpisodeEnemy` AS tblEpisodeEnemy ON tblEpisode.EpisodeId = tblEpisodeEnemy.EpisodeId
INNER JOIN `prueba_celes.tblEnemy` as tblEnemy ON tblEpisodeEnemy.EnemyId = tblEnemy.EnemyId
INNER JOIN `prueba_celes.tblDoctor` as tblDoctor ON tblEpisode.DoctorId = tblDoctor.DoctorId
WHERE DoctorName = "David Tennant" AND tblEnemy.EnemyId NOT IN (SELECT tblEnemy.EnemyId
FROM `prueba_celes.tblEpisode` AS tblEpisode
INNER JOIN `prueba_celes.tblEpisodeEnemy` AS tblEpisodeEnemy ON tblEpisode.EpisodeId = tblEpisodeEnemy.EpisodeId
INNER JOIN `prueba_celes.tblEnemy` as tblEnemy ON tblEpisodeEnemy.EnemyId = tblEnemy.EnemyId
INNER JOIN `prueba_celes.tblDoctor` as tblDoctor ON tblEpisode.DoctorId = tblDoctor.DoctorId
WHERE DoctorName != "David Tennant"
GROUP BY EnemyId)
GROUP BY tblEpisode.EpisodeId, tblEpisode.Title
ORDER BY tblEpisode.EpisodeId


De la consulta anterior resulta un listado de 32 episodios de David Tennant que no compartene enemigos con episodios de otros Doctores.
![](C:/Users/kaes1/Pictures/Screenshots/episode1.png)
![](C:/Users/kaes1/Pictures/Screenshots/episode2.png)


4.Eliminar registros duplicados para las cedula, basado en el Timestamp. Solo dejar el utlimo registro para cada cedula.

In [ ]:
SELECT tblCedula.Id, tblCedula.Cedula, TimeStamp
FROM `prueba_celes.tblCedula` as tblCedula
INNER JOIN (
SELECT Cedula, MAX(TimeStamp) as LastEntry
FROM `prueba_celes.tblCedula`
GROUP BY Cedula
) as tbl 
ON tblCedula.Cedula = tbl.Cedula AND tblCedula.TimeStamp = tbl.LastEntry
ORDER BY tblCedula.Cedula ASC

La tabla resultante es:
![](C:/Users/kaes1/Pictures/Screenshots/cedula.png)

## Python

1. Make a UDF which receives as a parameter a list of “Area/Location” strings. The result must be a single dataframe with
all responses from the endpoint: https://worldtimeapi.org/api/timezone/+Area+Location
e.g: https://worldtimeapi.org/api/timezone/America/New_York
Here you can find all timezones: https://worldtimeapi.org/timezones

The dataframe must have this columns:

    ID: Create a unique ID (not incremental ID)
    Year: Extract from “datetime”
    Month: Extract from “datetime”
    Day: Extract from “datetime”
    Time: “HH:MM:SS” from “datetime”
    Area: Extract from “timezone”
    Location: Extract from “timezone”
Iterate over at least 5 time zones with a sleep of 10 seconds after each iteration.


In [44]:
import pandas as pd
import requests
from time import sleep
from datetime import datetime
import uuid

lista = []

def get_timezone_location(locations_list):
  
  for location in locations_list:
    url = f"https://worldtimeapi.org/api/timezone/{location}"
    response = requests.get(url)

    if response.status_code == 200:
      response_data = response.json()
      datime = datetime.strptime(response_data.get("datetime"), "%Y-%m-%dT%H:%M:%S.%f%z")
      area = response_data.get("timezone").split("/")[0]
      location = response_data.get("timezone").split("/")[1]      
      year = datime.year
      month = datime.month
      day = datime.day
      tiempo = str(datime.time())[0:8]

      dict_time_location = {
        "Id": str(uuid.uuid4()),
        "Year": year,
        "Month": month,
        "Day": day,
        "Time": tiempo,
        "Area": area,
        "Location": location  
      }

    lista.append(dict_time_location)
    
    sleep(10)  
   
  df = pd.DataFrame(lista)
  
  return df



,Id,Year,Month,Day,Time,Area,Location
0,794939d9-4290-46bb-ad0c-b48d3c373808,2024,2,26,07:40:31,Asia,Bangkok
1,6a1e81f8-9752-4a3b-a3a8-27aac07fb804,2024,2,26,01:40:42,Europe,Rome
2,9b359dcf-3520-4453-a7fd-b0a61f264fbd,2024,2,25,14:40:52,Pacific,Honolulu
3,9d4e49fc-9161-4462-93b0-34197c6918a3,2024,2,25,21:41:03,America,Sao_Paulo
4,be70fa88-6311-4868-8be8-fc6d52093d0a,2024,2,26,02:41:14,Africa,Cairo


In [71]:
locations = ["Asia/Bangkok", "Europe/Rome", "Pacific/Honolulu", "America/Sao_Paulo", "Africa/Cairo"]

df_locations = get_timezone_location(locations)
df_locations

,Id,Year,Month,Day,Time,Area,Location
0,794939d9-4290-46bb-ad0c-b48d3c373808,2024,2,26,07:40:31,Asia,Bangkok
1,6a1e81f8-9752-4a3b-a3a8-27aac07fb804,2024,2,26,01:40:42,Europe,Rome
2,9b359dcf-3520-4453-a7fd-b0a61f264fbd,2024,2,25,14:40:52,Pacific,Honolulu
3,9d4e49fc-9161-4462-93b0-34197c6918a3,2024,2,25,21:41:03,America,Sao_Paulo
4,be70fa88-6311-4868-8be8-fc6d52093d0a,2024,2,26,02:41:14,Africa,Cairo
5,be050f0b-3cef-4ff1-b6e9-e2dfbc37d403,2024,2,26,18:55:20,Asia,Bangkok
6,fd46d9f5-41f7-4970-a66e-602f5e95baee,2024,2,26,12:55:31,Europe,Rome
7,a999caed-abd2-438f-8f98-90642fb72f8c,2024,2,26,01:55:42,Pacific,Honolulu
8,93d5571c-6abd-4b9b-b06c-c974ec2352b0,2024,2,26,08:55:52,America,Sao_Paulo
9,aa48b3a7-c8f7-419c-96a7-18cf5d0a9d05,2024,2,26,13:56:03,Africa,Cairo


2. Crear una función que permita convertir números enteros en números romanos. (Para este caso se limita la función para que solo pueda convertir números de 4 digitos como maximo)

In [69]:
def get_romano(num_entero):
    
    num_romanos = {"M": 1000,
                "CM": 900,
                "D": 500,
                "CD": 400,
                "C": 100,
                "XC": 90,
                "L": 50,
                "XL": 40,
                "X": 10,
                "IX": 9,
                "V": 5,
                "IV": 4,
                "I": 1    
    }
    
    if (len(str(num_entero))) <= 4:      
        num_romano = str()
        
        for romano, entero in num_romanos.items():
            while (num_entero >= entero):
                num_romano = num_romano + romano
                num_entero = num_entero - entero
                
    else:
        num_romano = "Solo se pueden convertir números de 4 o menos digitos"
        
    return num_romano
        
        
get_romano(101)      

'CI'